In [ ]:
"app.ipynb"

In [ ]:
import time
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

In [ ]:
# Set up Selenium WebDriver
chromedriver_path = "./chromedriver-32.exe"
service = Service(chromedriver_path)

# Set the path to the Portable Chrome executable
chrome_exe_path = "./chrome-win32/chrome.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = chrome_exe_path
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--blink-settings=imagesEnabled=false")

driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# for start_url in START_URL_LIST:
start_url = "https://airtable.com/embed/shrCf4smsrEmVyebB?backgroundColor=gray&viewControls=on"

In [ ]:
driver.get(start_url)
time.sleep(5)

In [ ]:
main_element = driver.find_element(By.CSS_SELECTOR, "div.light-scrollbar")

In [ ]:
# Remove the 'light-scrollbar' class using JavaScript
driver.execute_script("arguments[0].classList.remove('light-scrollbar');", main_element)
time.sleep(1)

In [ ]:
# Function to get the current scroll position and scroll height
def get_scroll_info(element):
    return driver.execute_script("""
        return {
            scrollTop: arguments[0].scrollTop,
            scrollHeight: arguments[0].scrollHeight,
            clientHeight: arguments[0].clientHeight
        };
    """, element)

In [ ]:
# Scroll down 400px at a time until the bottom is reached
while True:
    elements = main_element.find_elements(By.CSS_SELECTOR, "div.galleryCardContainer")
    for element in elements:
        element_list = element.find_elements(By.XPATH, ".//div[@role='presentation']/div/div")
        if len(element_list) == 5:
            companyName = element_list[2].text.strip() or "N/A"
            otherLink = element_list[2].find_element(By.TAG_NAME, "a").get_attribute("href") or "#"
            companySolution = element_list[3].find_element(By.CSS_SELECTOR, "div.cellContainer span").text.strip() or "N/A"
            # companyWebsite = element_list[4].find_element(By.CSS_SELECTOR, "div.cellContainer").text.strip() or "#"
            print(companyName, companySolution, otherLink)
        else:
            print(f"Warning! The size of element-list is {len(element_list)}")
    
    scroll_info = get_scroll_info(main_element)
    scroll_top = scroll_info['scrollTop']
    scroll_height = scroll_info['scrollHeight']
    client_height = scroll_info['clientHeight']
    
    # Break the loop if we've reached the bottom
    if scroll_top + client_height >= scroll_height:
        break
    
    # Scroll down 400px
    driver.execute_script("arguments[0].scrollBy(0, 400);", main_element)
    time.sleep(0.5)